# 项目简介
使用2021年9月嵊山站（浙江省舟山市嵊泗县）气象要素站点观测资料，在2114号台风“灿都”影响下，进行消除气温影响的气压和风速的的偏相关分析。本案例笔者使用的是python第三方库pandas中的相关函数，读者也可以尝试其他方法进行计算。

## 数据介绍
数据来源：国家海洋科学数据中心-实测数据-海洋水文-中国台站观测数据
http://mds.nmdis.org.cn/pages/dataViewDetail.html?dataSetId=4

时间分辨率：1h   站点：SSN嵊山站

*直接下载的文件为txt文件，已将其整理并转换为csv文件

## 研究思路：
本案例中，气压可以在一定程度上反映台风及其周围的中小尺度对流系统等对观测站点的影响（台风-低压），同时气温的日变化&中长期变化趋势等也会对气压造成影响。为了更准确地研究气压和风速的关系，可计算消除气温影响下，气压和风速的偏相关系数。

## 配套视频链接：

In [1]:
import pandas as pd
from pandas import DataFrame
import datetime as dt
import pytz

In [2]:
df1 = pd.read_csv(r'C:\Users\LULU\Desktop\TC\data\ocean_data\SSN2109.csv')
df1['date'] = pd.to_datetime(df1['date'])

#筛选指定时段
day1 = dt.datetime.strptime('2021-9-18 19','%Y-%m-%d %H').replace(tzinfo=pytz.timezone('UTC'))
day2 = dt.datetime.strptime('2021-9-7 0','%Y-%m-%d %H').replace(tzinfo=pytz.timezone('UTC'))
df2 = df1[(df1['date']<day1)&(df1['date']>day2)]

df = DataFrame({'Temperature':df2['T'],'Wind_speed':df2['wind_speed'],'Air_pressure':df2['P']})

## 0.【展示】站点观测数据时间序列（气温、风速、气压、台风和站点的距离）

![站点观测数据时间序列（气温、风速、气压、台风离该站的距离）](./ShengShan.jpg)

## 1.查看数据

In [3]:
print(df)

     Temperature  Wind_speed  Air_pressure
153         24.8        10.2        1007.3
154         25.1        10.4        1007.7
155         25.5         9.8        1008.2
156         25.9         9.6        1008.4
157         26.1         8.9        1008.3
..           ...         ...           ...
430         24.1         1.1        1009.3
431         24.0         2.4        1010.0
432         23.9         2.4        1010.2
433         23.6         2.4        1010.3
434         23.6         2.3        1010.2

[282 rows x 3 columns]


## 2.三个变量两两之间的相关系数矩阵

In [4]:
df.corr()

,Temperature,Wind_speed,Air_pressure
Temperature,1.000000,-0.415792,0.184012
Wind_speed,-0.415792,1.000000,-0.726433
Air_pressure,0.184012,-0.726433,1.000000


## 3.在三个变量中，消除变量【气温】影响下，变量【气压】和【风速】之间的一阶偏相关系数

In [5]:
r_wa=df['Wind_speed'].corr(df['Air_pressure'])
r_at=df['Air_pressure'].corr(df['Temperature'])
r_wt=df['Wind_speed'].corr(df['Temperature'])
r_wa_t=(r_wa-r_at*r_wt)/(((1-r_at**2)**0.5)*((1-r_wt**2)**0.5))
print('气压和风速的相关系数：',r_wa)
print('消除气温影响下，气压和风速的偏相关系数：',r_wa_t)

气压和风速的相关系数： -0.7264329280078583
消除气温影响下，气压和风速的偏相关系数： -0.7270392344821902


## 4.解释现象
影响气温变化的因素是多尺度的。将气温时间序列数据进行滤波可发现，台风对气温的冷却效应主要体现在短期分量的作用。由于本案例中气温和气压的相关系数相对不高，故在消除气温影响下，气压和风速的相关性只是略强了一些。